In [70]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 실제 분포
beta_true = 1.0 / (0.3) ** 2  # 분산: 0.3
alpha = 1.0 / (100) ** 2  # 사전 분포 분산

beta = beta_true

M = 9  # 다항식 차수
N_list = [4, 5, 10, 100]

# 트레이닝 셋 {x_n,y_n} (n=1...N) 생성 함수
def create_training_set(num):
    dataset = pd.DataFrame(columns=['x','y'])
    for i in range(num):
        x = float(i) / float(num-1)  # x의 범위: [0,1]
        y = np.sin(2.0 * np.pi * x) + np.random.normal(scale=0.3)
        dataset = dataset.append(pd.Series([x,y], index = ['x','y']),
                                 ignore_index=True)
    return dataset

In [71]:
# dataset =
phis = pd.DataFrame() # (size=(N*M+1))

num = 4
m = 4

training_set = create_training_set(num)
phis = pd.DataFrame()
for i in range(0,m+1):
    p = training_set.y ** i
    p.name = "x^%d" % i
    phis = pd.concat([phis,p],axis=1)

In [82]:
#분산 S 계산
phiphi_sum = 0
for _,line in phis.iterrows():
    phi = line[:,np.newaxis] # (size=(M+1,1))
    phiphi = np.dot(phi,phi.T) # (size=(M+1,M+1))
    phiphi_sum += phiphi
s_inv = alpha * np.identity(m+1) + beta*phiphi_sum
s = np.linalg.inv(s_inv)

t = training_set.y
# 평균 m(x) 계산
def calculate_mean(x):
    phi_x = pd.DataFrame([x ** i for i in range(0, m + 1)])
    sigma_t_phi = np.multiply(t[:,np.newaxis],phis)
    print(np.dot(phi_x.T, s).shape)
    print((sigma_t_phi).shape)
    # return (beta * np.dot(np.dot(phi_x.T, s), pd.DataFrame(sigma_t_phi))).flatten()

calculate_mean(0)

(1, 5)
(4, 5)


<ipython-input-82-9f4962b25fd6>:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  phi = line[:,np.newaxis] # (size=(M+1,1))
<ipython-input-82-9f4962b25fd6>:14: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  sigma_t_phi = np.multiply(t[:,np.newaxis],phis)
